In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
url = 'http://effatha.pl/slownik.html'

In [4]:
index = requests.get(url)

In [5]:
bs = BeautifulSoup(index.text, 'html5lib')

In [6]:
entries = bs.find(name='div', attrs={'id': 'slownik'}).find_all(['li'])

In [7]:
from tqdm import tqdm_notebook

In [8]:
# entries_clean = []
# for e in tqdm_notebook(entries):
#     if e.find('h4'):
#         link = e.find('a')
#         if link and link.attrs['href'].startswith('slownik'):
#             entries_clean.append(str(link))

In [9]:
# len(entries_clean)

In [10]:
import re

In [11]:
# entries_clean

In [12]:
# t = entries_clean[0]

In [13]:
# re.findall(r'href="(\S+)"', str(entries_clean[0]))

In [14]:
def get_yt_url(e):
    href = re.findall(r'href="(\S+)"', str(e))[0]
    data = requests.get('http://effatha.pl/' + href).text
    if data:
        iframe = re.findall(r'iframe\s*src="(\S+)"', str(data))[0]
#         iframe = BeautifulSoup(data,'html5lib').find('iframe')
        return iframe
    else:
        print('error')

In [15]:
# urls = [get_yt_url(e) for e in tqdm_notebook(entries_clean)]

In [16]:
# %time movies = p.map(get_yt_url, entries_clean[:10])    

In [17]:
import os

In [18]:
movies = os.listdir('./movies/')

In [19]:
import unicodedata

In [20]:
dictionary = dict([(unicodedata.normalize('NFC', p.split('-')[0].lower()), p) for p in movies])

In [72]:
[k for k in dictionary.keys() if k.startswith('sp')]

['spółka',
 'sprawdzać',
 'spóźnienie',
 'sport',
 'sprytny',
 'specjalność',
 'sprzedawać',
 'spierać się',
 'spragniony',
 'spodnie',
 'spuchnąć',
 'sprzątać',
 'sprzedać',
 'spacer',
 'spokojny',
 'sparaliżowany',
 'spódnica',
 'sposób',
 'spółdzielnia',
 'specjalny',
 'społeczeństwo',
 'spać',
 'społeczny',
 'spotkanie',
 'spór',
 'spowiedź',
 'spirytus',
 'sprawiedliwość',
 'sprawa',
 'sport2']

In [22]:
import subprocess

In [23]:
import pexpect
def lemmatize(word):
    child = pexpect.spawn('morfeusz_analyzer')
    child.expect('\)\r\n')
    child.sendline(word)
    child.readline()
    response = child.readline()
    return response.decode('utf-8').split(',')[3].split(':')[0]

In [24]:
lemmatize('mam')

'mamić'

In [25]:
import Levenshtein

In [73]:
def find_synonym(word):
    for k,p in dictionary.items():
        if Levenshtein.ratio(k, word) >= 0.7:
            return k,p
    synonyms = get_synonyms(word)
    for syn in synonyms:
        for k,p in dictionary.items():
            if Levenshtein.ratio(k, syn) >= 0.7:
                return k,p
    return None,None

In [74]:
lemmatize('opóźniony')

'opóźnić'

In [75]:
from ivr.wdnet import is_known, get_synonyms, are_synonyms

In [104]:
sent = 'pociąg relacji Warszawa Gdańsk jest opóźniony o 10 minut'

In [105]:
from moviepy.editor import *

In [106]:
from many_stop_words import get_stop_words

In [107]:
stopwords = get_stop_words('pl')

In [108]:
def word2clip(word):
    if word in stopwords:
        return
    if word not in dictionary:
        lemmatized = lemmatize(word) or word
    else:
        lemmatized = word
    clip = dictionary.get(lemmatized, None)
    if clip:
        print('original: ', word, 'lemmatized: ', lemmatized, 'clip', clip, type(clip))
        return VideoFileClip(os.path.join('movies', clip))
    else:
        synonym, clip = find_synonym(word)
        print('original: ', word, 'synnonym: ', synonym, 'clip', clip, type(clip))
        if clip:
            return VideoFileClip(os.path.join('movies', clip))

In [121]:
def sent2movie(sent):
    clips = []
    for w in sent.split():
        clip = word2clip(w)
        if clip:
            clips.append(clip.resize((460,460)))
    print(clips)
    return concatenate_videoclips(clips)

In [122]:
movie = sent2movie(sent)

original:  pociąg lemmatized:  pociąg clip pociąg-EMvnOAZopLo.mp4 <class 'str'>
original:  relacji synnonym:  None clip None <class 'NoneType'>
original:  Warszawa lemmatized:  warszawa clip Warszawa-aO5yT8agwfQ.mp4 <class 'str'>
original:  Gdańsk synnonym:  gdańsk clip Gdańsk-5t9y_rv_PK4.mp4 <class 'str'>
original:  opóźniony synnonym:  spóźnienie clip spóźnienie-MVbLQ4vX-7I.mp4 <class 'str'>
original:  10 lemmatized:  10 clip 10-y_tUPPPcJfc.mp4 <class 'str'>
original:  minut lemmatized:  minuta clip minuta-nnyW_39XwsY.mp4 <class 'str'>
[<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x110d5b2b0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x110d5bac8>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x110d61208>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x110d61278>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x10cd60c88>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x10cd60908>]


In [123]:
movie.duration

19.0

In [124]:
# movie = movie.speedx(2)

In [125]:
movie.duration

19.0

In [127]:
movie.write_videofile('./tmp.mp4')

[MoviePy] >>>> Building video ./tmp.mp4
[MoviePy] Writing audio in tmpTEMP_MPY_wvf_snd.mp3


100%|██████████| 419/419 [00:00<00:00, 764.02it/s]

[MoviePy] Done.
[MoviePy] Writing video ./tmp.mp4



100%|█████████▉| 475/476 [00:07<00:00, 63.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./tmp.mp4 

